In [1]:
# Loading libraries
%matplotlib inline
import os
import os.path
from netCDF4 import Dataset, num2date, date2num
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc 
import fnmatch
from cycler import cycler
import pandas as pd
from pandas import read_csv, to_datetime, DataFrame
import datetime
import time
import matplotlib.dates as mdates

from matplotlib.path import Path
from matplotlib.patches import PathPatch
from mpl_toolkits.basemap import Basemap # Basemap pour les cartes

### TEM ponderation prof
from datetime import datetime, timedelta

In [2]:
# Chemin des fichiers à utiliser
file_path = './ANNUAL_MEANS_BestAjustRivE_BoostW_BOUCLE/'
full_list = os.listdir(file_path)
full_list.sort()
#full_list

In [3]:
# LOADING GRID FILE AND VARIABLES
!pwd
grid_file = '../DATA/grille.nc' 
print 'file exist:', os.path.isfile(grid_file)

f = Dataset(grid_file) # open netcdf
# extracting variables
lon_t = f.variables['longitude_t'][:] # extracting lon data
lat_t = f.variables['latitude_t'][:] # extracting lat data
#depth_t = f.variables['depth_t'][:]
dz_t = f.variables['dz_t'][:]
bathy = f.variables['h_w'][:]
print 'Bathy :', np.shape(bathy), 'max :', np.max(bathy), 'min :', np.min(bathy)

f.close()

new_bath = np.ma.masked_where(bathy<200, bathy)

/home/dosa/MON_ENV_PYTHON/ARTICLE_IAV
file exist: True
Bathy : (160, 394) max : 5054.49 min : 20.5226


In [9]:
# # NEW VERSION NUTRIENTS + ecriture ncdf

# For 196 m
count = 0
time_s = np.zeros([len(full_list)]) # replace file_list here

for file in full_list:
    print file
    
    # extracting variables
    f = Dataset(file_path+file) # open netcdf
    chl_tot_196 = f.variables['chl_tot'][0,:,:,:] # extracting # [0,30:43,:,:]
    f.close()
    
    # Creating empty arrays to store integrated values
    vars()['chl_tot_196_'+str(file[0:4])] = np.zeros([160,394])
    vars()['chl_tot_196_'+str(file[0:4])][:] = np.nan
    
    # At each lvl
    for z in np.arange(31,43):
        # taking this particular lvl, masking it to avoid useless computations
        vars()['chl_tot_196_lvl_'+str(z)] = np.ma.masked_where(bathy<200, chl_tot_196[z,:,:])
            
    # for each (2D) point
    for i in np.arange(0,160): # np.arange(0,160):
        for j in np.arange(0,394):  #np.arange(0,394):
            if np.ma.is_masked(new_bath[i,j]) == False: # to avoid useless points
                #print 'i ,j =', i, j
                chl_tot_ij = 0
                chl_tot_this_lvl = 0

                for z in np.arange(31,43): # there we need to consider one less lvl
                    #print 'z =', z
                    #print 'dz_t =', dz_t[z,i,j]
                    #print 'chl_tot =', vars()['chl_tot_196_lvl_'+str(z)][i,j]
                    chl_tot_this_lvl = vars()['chl_tot_196_lvl_'+str(z)][i,j] * dz_t[z,i,j]
                    #print 'chl_tot_this_lvl =', chl_tot_this_lvl
                    chl_tot_ij = np.nansum([chl_tot_ij, chl_tot_this_lvl])
                    #print 'chl_tot_ij =', chl_tot_ij
                    #print ' '
                    
                # Nuts => no need to weigh by depth                

                # now putting our int chl value in the output array
                vars()['chl_tot_196_'+str(file[0:4])][i,j] = chl_tot_ij
                #print ' '
                               
    #print ' '
    print 'Creating ncdf'
    ### Creating NetCDF files for our outputs! ###
    dates = []
    
    mask_alex = Dataset('./NC_INT_OUTPUTS_ALEX/ChlTot_Int150_'+str(file[0:4])+'.nc', 'w', format='NETCDF4_CLASSIC')
    
    # creating dimensions
    time = mask_alex.createDimension('time',1)
    ni_t = mask_alex.createDimension('nj_t', 160) 
    nj_t = mask_alex.createDimension('ni_t', 394)
    
    # creating variable with dimensions
    #times = mask_alex.createVariable('time', np.int32, ('time',))  # float64
    times = mask_alex.createVariable('time', np.float64, ('time',))  # float64
    times.units = 'seconds since 1970-06-15 12:10:01' 
    times.calendar = 'gregorian'
    
    dates.append(datetime(int(file[0:4]), 10, 3, 12, 10, 1)) # according to ncdf files already existing 
    times[:] = date2num(dates, units = times.units, calendar = times.calendar)
    #times[:] = date2num(dates)
    #print dates

    mask_t = mask_alex.createVariable('ChlTot_Int_0-151', np.float32, ('time', 'nj_t', 'ni_t'), fill_value=-9999) #'time', 
    mask_t.units = 'mmol.m-2'
    #mask_t.units = 'degrees_Celsius'
    
    #print 'temp variable:', mask_alex.variables['Tem_Int_0-151']
    
    mask_t[:] =vars()['chl_tot_196_'+str(file[0:4])][:] # putting our df inside our variable
    mask_alex.close() # writing the netcdf file!

1970.nc
Creating ncdf
1971.nc
Creating ncdf
1972.nc
Creating ncdf
1973.nc
Creating ncdf
1974.nc
Creating ncdf
1975.nc
Creating ncdf
1976.nc
Creating ncdf
1977.nc
Creating ncdf
1978.nc
Creating ncdf
1979.nc
Creating ncdf
1980.nc
Creating ncdf
1981.nc
Creating ncdf
1982.nc
Creating ncdf
1983.nc
Creating ncdf
1984.nc
Creating ncdf
1985.nc
Creating ncdf
1986.nc
Creating ncdf
1987.nc
Creating ncdf
1988.nc
Creating ncdf
1989.nc
Creating ncdf
1990.nc
Creating ncdf
1991.nc
Creating ncdf
1992.nc
Creating ncdf
1993.nc
Creating ncdf
1994.nc
Creating ncdf
1995.nc
Creating ncdf
1996.nc
Creating ncdf
1997.nc
Creating ncdf
1998.nc
Creating ncdf
1999.nc
Creating ncdf
2000.nc
Creating ncdf
2001.nc
Creating ncdf
2002.nc
Creating ncdf
2003.nc
Creating ncdf
2004.nc
Creating ncdf
2005.nc
Creating ncdf
2006.nc
Creating ncdf
2007.nc
Creating ncdf
2008.nc
Creating ncdf
2009.nc
Creating ncdf
2010.nc
Creating ncdf
2011.nc
Creating ncdf
